# Analysing the correlation between score and settings

In [ ]:
import matplotlib.pyplot as plt
import yaml
import os
import csv
from collections import defaultdict

In [ ]:
def combine_dicts(dict1, dict2):
    combine = dict1.copy()
    for key, value in dict2.items():
        if key in combine and isinstance(combine[key], dict):
            combine[key] = combine_dicts(combine[key], value)
        else:
            combine[key] = value
    return combine

In [ ]:
DEFAULT_CONFIG_PATH = "./default_config.yaml"

with open(DEFAULT_CONFIG_PATH, "r") as file:
    default_config = yaml.safe_load(file)

DATA_FOLDER = "./data"
all_files = {
    os.path.splitext(f)[0]
    for f in os.listdir(DATA_FOLDER)
    if os.path.isfile(os.path.join(DATA_FOLDER, f))
}

print(f"Found {len(all_files)} submissions.")

submission_data = {}

for submission_name in all_files:
    run_config_file = os.path.join(DATA_FOLDER, f"{submission_name}.yaml")
    run_submission_file = os.path.join(DATA_FOLDER, f"{submission_name}.csv")

    with open(run_config_file, "r") as run_file:
        run_config = combine_dicts(default_config, yaml.safe_load(run_file))

    with open(run_submission_file, mode="r") as submission_file:
        csv_reader = csv.DictReader(submission_file)
        run_submission = {row["quadrat_id"]: row["species_ids"] for row in csv_reader}

    submission_data[submission_name] = {
        "config": run_config,
        "submission": run_submission,
    }

In [ ]:
# Filter submissions we don't want to see

# submission_data = {k: v for k, v in submission_data.items() if v['config']['models']['checkpoint_file'] == '5heads_1layer'}

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    if data["config"]["prediction"]["method"] != "top_k_tile":
        continue
    scores[data["config"]["prediction"]["top_k_tile"]["k"]].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(scores.values(), positions=scores.keys())
plt.xlabel("k")
plt.ylabel("Score")
plt.title("Boxplot of Scores by top k")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[sorted(data["config"]["prediction"]["tiling"]["scales"]).__str__()].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("scales")
plt.ylabel("Score")
plt.title("Boxplot of Scores by scales")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[sorted(data["config"]["prediction"]["tiling"]["overlaps"]).__str__()].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("overlaps")
plt.ylabel("Score")
plt.title("Boxplot of Scores by overlaps")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    if data["config"]["prediction"]["method"] != "top_k_tile":
        continue
    scores[data["config"]["prediction"]["top_k_tile"]["min_score"]].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("min_score")
plt.ylabel("Score")
plt.title("Boxplot of Scores by top k min score")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    if data["config"]["prediction"]["method"] != "BMA":
        continue
    scores[data["config"]["prediction"]["BMA"]["z_score_threshold"]].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

if len(scores) != 0:
    plt.boxplot(
        scores.values(),
        positions=range(len(scores.keys())),
        tick_labels=list(scores.keys()),
    )
    plt.xlabel("z_score")
    plt.ylabel("Score")
    plt.title("Boxplot of Scores by BMA z-score")
    plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[data["config"]["prediction"]["filter_genus"]].append(data["config"]["score"])

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("filter_genus")
plt.ylabel("Score")
plt.title("Boxplot of Scores by filter_genus")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[data["config"]["prediction"]["use_genus_and_family"]].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("use_genus_and_family")
plt.ylabel("Score")
plt.title("Boxplot of Scores by use_genus_and_family")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[data["config"]["models"]["checkpoint_file"]].append(data["config"]["score"])

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("checkpoint_file")
plt.ylabel("Score")
plt.title("Boxplot of Scores by checkpoint_file")
plt.show()

In [ ]:
scores = defaultdict(list)
for submission_name, data in submission_data.items():
    scores[data["config"]["prediction"]["filter_species_threshold"]].append(
        data["config"]["score"]
    )

scores = dict(sorted(scores.items()))

plt.boxplot(
    scores.values(),
    positions=range(len(scores.keys())),
    tick_labels=list(scores.keys()),
)
plt.xlabel("filter_species_threshold")
plt.ylabel("Score")
plt.title("Boxplot of Scores by filter_species_threshold")
plt.show()

In [ ]:
print(data["submission"])

prediction_lengths = {}
for submission_name, data in submission_data.items():
    prediction_lengths[data["config"]["score"]] = [
        len(pred.split(" ")) for pred in data["submission"].values()
    ]

print(prediction_lengths)

prediction_lengths = dict(sorted(prediction_lengths.items()))

plt.boxplot(
    prediction_lengths.values(),
    positions=range(len(prediction_lengths.keys())),
    tick_labels=list(prediction_lengths.keys()),
)
plt.xlabel("score")
plt.ylabel("len of predictions")
plt.yscale("log")
plt.title("Boxplot prediction lengths by score")
plt.show()